In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [3]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution, kl_reg

# Data

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 17:26:24.214613 140268265989952 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 17:26:24.221558 140268265989952 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 17:26:24.226097 140268265989952 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



# GCN (baseline)

In [10]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 17:26:24.234217 140268265989952 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 17:26:24.240765 140268265989952 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 17:26:24.249119 140268265989952 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [11]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 17:26:24.290334 140268265989952 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]                  
                                                                 input_1[0][0]                    
__________

In [13]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

W1110 17:26:24.366671 140268265989952 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9311 train_acc= 0.2929 val_loss= 1.9319 val_acc= 0.3500 time= 1.0749
Epoch: 0002 train_loss= 1.9151 train_acc= 0.2929 val_loss= 1.9171 val_acc= 0.3500 time= 0.0192
Epoch: 0003 train_loss= 1.8975 train_acc= 0.2929 val_loss= 1.9011 val_acc= 0.3500 time= 0.0238
Epoch: 0004 train_loss= 1.8792 train_acc= 0.2929 val_loss= 1.8845 val_acc= 0.3500 time= 0.0199
Epoch: 0005 train_loss= 1.8608 train_acc= 0.2929 val_loss= 1.8679 val_acc= 0.3500 time= 0.0253
Epoch: 0006 train_loss= 1.8418 train_acc= 0.2929 val_loss= 1.8512 val_acc= 0.3500 time= 0.0241
Epoch: 0007 train_loss= 1.8228 train_acc= 0.2929 val_loss= 1.8347 val_acc= 0.3500 time= 0.0196
Epoch: 0008 train_loss= 1.8039 train_acc= 0.2929 val_loss= 1.8187 val_acc= 0.3500 time= 0.0189
Epoch: 0009 train_loss= 1.7853 train_acc= 0.2929 val_loss= 1.8036 val_acc= 0.3500 time= 0.0189
Epoch: 0010 train_loss= 1.7672 train_acc= 0.2929 val_loss= 1.7896 val_acc= 0.3500 time= 0.0216
Epoch: 0011 train_loss= 1.7509 train_acc= 0.2929 v

Epoch: 0088 train_loss= 0.7507 train_acc= 0.9000 val_loss= 1.0426 val_acc= 0.7800 time= 0.0186
Epoch: 0089 train_loss= 0.7427 train_acc= 0.9000 val_loss= 1.0347 val_acc= 0.7833 time= 0.0257
Epoch: 0090 train_loss= 0.7350 train_acc= 0.9000 val_loss= 1.0278 val_acc= 0.7833 time= 0.0237
Epoch: 0091 train_loss= 0.7277 train_acc= 0.9000 val_loss= 1.0209 val_acc= 0.7800 time= 0.0206
Epoch: 0092 train_loss= 0.7204 train_acc= 0.9000 val_loss= 1.0136 val_acc= 0.7833 time= 0.0198
Epoch: 0093 train_loss= 0.7135 train_acc= 0.9071 val_loss= 1.0067 val_acc= 0.7767 time= 0.0240
Epoch: 0094 train_loss= 0.7067 train_acc= 0.9071 val_loss= 1.0010 val_acc= 0.7733 time= 0.0184
Epoch: 0095 train_loss= 0.6996 train_acc= 0.9143 val_loss= 0.9966 val_acc= 0.7800 time= 0.0235
Epoch: 0096 train_loss= 0.6929 train_acc= 0.9214 val_loss= 0.9932 val_acc= 0.7800 time= 0.0235
Epoch: 0097 train_loss= 0.6868 train_acc= 0.9214 val_loss= 0.9901 val_acc= 0.7800 time= 0.0255
Epoch: 0098 train_loss= 0.6807 train_acc= 0.9214 v

Epoch: 0184 train_loss= 0.3990 train_acc= 0.9714 val_loss= 0.7784 val_acc= 0.8100 time= 0.0245
Epoch: 0185 train_loss= 0.3968 train_acc= 0.9714 val_loss= 0.7764 val_acc= 0.8067 time= 0.0237
Epoch: 0186 train_loss= 0.3951 train_acc= 0.9714 val_loss= 0.7755 val_acc= 0.8067 time= 0.0233
Epoch: 0187 train_loss= 0.3939 train_acc= 0.9714 val_loss= 0.7753 val_acc= 0.8067 time= 0.0189
Epoch: 0188 train_loss= 0.3924 train_acc= 0.9714 val_loss= 0.7754 val_acc= 0.8067 time= 0.0248
Epoch: 0189 train_loss= 0.3909 train_acc= 0.9714 val_loss= 0.7759 val_acc= 0.8067 time= 0.0233
Epoch: 0190 train_loss= 0.3889 train_acc= 0.9786 val_loss= 0.7762 val_acc= 0.8067 time= 0.0244
Epoch: 0191 train_loss= 0.3872 train_acc= 0.9786 val_loss= 0.7751 val_acc= 0.8067 time= 0.0196
Epoch: 0192 train_loss= 0.3862 train_acc= 0.9857 val_loss= 0.7763 val_acc= 0.8067 time= 0.0291
Epoch: 0193 train_loss= 0.3854 train_acc= 0.9786 val_loss= 0.7761 val_acc= 0.8100 time= 0.0236
Epoch: 0194 train_loss= 0.3836 train_acc= 0.9786 v

In [14]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.8003
accuracy = 0.8050


# Gaussian Graph Convolution

In [15]:
H = Dropout(0.5)(X_in)
H1 = GaussianGraphConvolution(16,
    is_first=True,
    activation='relu',
)([H]+G)
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    activation='softmax',
)(H1+G)

W1110 17:26:29.839422 140268265989952 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/ggcl.py:28: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
W1110 17:26:29.840713 140268265989952 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/distributions/normal.py:160: Distribution.__init__ (from tensorflow.python.ops.distributions.distribution) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update

In [16]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
# model.add_loss()

In [17]:
model.add_loss(kl_reg(*H1), H1)

W1110 17:26:29.879734 140268265989952 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/losses.py:27: kl_divergence (from tensorflow.python.ops.distributions.kullback_leibler) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [18]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
gaussian_graph_convolution_1 (G [(None, 16), (None,  45856       dropout_3[0][0]                  
                                                                 input_1[0][0]                    
__________

In [19]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

Epoch: 0001 train_loss= 1.9409 train_acc= 0.3929 val_loss= 1.9429 val_acc= 0.2700 time= 0.4139
Epoch: 0002 train_loss= 1.9334 train_acc= 0.5429 val_loss= 1.9387 val_acc= 0.4200 time= 0.0234
Epoch: 0003 train_loss= 1.9242 train_acc= 0.6786 val_loss= 1.9324 val_acc= 0.5500 time= 0.0250
Epoch: 0004 train_loss= 1.9131 train_acc= 0.7143 val_loss= 1.9257 val_acc= 0.5933 time= 0.0282
Epoch: 0005 train_loss= 1.9012 train_acc= 0.6857 val_loss= 1.9177 val_acc= 0.5800 time= 0.0231
Epoch: 0006 train_loss= 1.8897 train_acc= 0.6857 val_loss= 1.9095 val_acc= 0.5600 time= 0.0193
Epoch: 0007 train_loss= 1.8765 train_acc= 0.6286 val_loss= 1.8994 val_acc= 0.5400 time= 0.0274
Epoch: 0008 train_loss= 1.8617 train_acc= 0.6643 val_loss= 1.8907 val_acc= 0.5600 time= 0.0226
Epoch: 0009 train_loss= 1.8451 train_acc= 0.6500 val_loss= 1.8810 val_acc= 0.5133 time= 0.0236
Epoch: 0010 train_loss= 1.8308 train_acc= 0.6357 val_loss= 1.8671 val_acc= 0.5267 time= 0.0183
Epoch: 0011 train_loss= 1.8154 train_acc= 0.6286 v

Epoch: 0097 train_loss= 0.2553 train_acc= 0.9786 val_loss= 0.6791 val_acc= 0.8267 time= 0.0235
Epoch: 0098 train_loss= 0.2508 train_acc= 0.9786 val_loss= 0.6740 val_acc= 0.8300 time= 0.0204
Epoch: 0099 train_loss= 0.2436 train_acc= 0.9786 val_loss= 0.6698 val_acc= 0.8233 time= 0.0281
Epoch: 0100 train_loss= 0.2370 train_acc= 0.9786 val_loss= 0.6646 val_acc= 0.8200 time= 0.0186
Epoch: 0101 train_loss= 0.2294 train_acc= 0.9786 val_loss= 0.6611 val_acc= 0.8200 time= 0.0183
Epoch: 0102 train_loss= 0.2229 train_acc= 0.9786 val_loss= 0.6569 val_acc= 0.8200 time= 0.0189
Epoch: 0103 train_loss= 0.2171 train_acc= 0.9786 val_loss= 0.6535 val_acc= 0.8200 time= 0.0184
Epoch: 0104 train_loss= 0.2103 train_acc= 0.9786 val_loss= 0.6488 val_acc= 0.8233 time= 0.0227
Epoch: 0105 train_loss= 0.2043 train_acc= 0.9786 val_loss= 0.6454 val_acc= 0.8233 time= 0.0194
Epoch: 0106 train_loss= 0.2014 train_acc= 0.9786 val_loss= 0.6425 val_acc= 0.8233 time= 0.0194
Epoch: 0107 train_loss= 0.1938 train_acc= 0.9857 v

In [20]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.6229
accuracy = 0.8030


# Attacks